# Stations Over Time
This notebook generates a timelapse of station availability over time. The `stations.csv` file is manually populated, and used to generate a `stations-over-time.csv` file. 

In [ ]:
# import conjugate_map.conj_calc as conj
import datetime as dt
import pandas as pd
import plotly.express as px

import geopandas
# import gpxpy
# import gpxpy.gpx
# import json

import logging
logger = logging.getLogger(__name__)

In [ ]:
stations = pd.read_csv('stations_AntarcticdatacoverageFIgure.csv')
stations["End"] = stations["Estimated End"]
stations["Category"] = stations["Category"].fillna("Base")
stations["Network"] = stations["Network"].fillna("Base")
stations["End"] = stations["End"].fillna(2034) # assume that all stations will last unless noted otherwise
stations["Start"] = stations["Start"].fillna(1980) # assume that all stations date back to 1980 unless noted otherwise
stations = pd.DataFrame(stations)


# Restrict to only the networks of interest:
# stations = stations[stations.Network.isin(["MACCS", "AALPIP"])]
# pd.unique(stations.Institution)

In [ ]:
stations

In [ ]:
# Preallocate:
stations_over_time = pd.DataFrame()


for year in [2007, 2025, 2032]:
    try:
        print("Computing for " + str(year) + "...................................................................")
        # run conjcalc for the year of interest
        # data = conj.conjcalc(stations, latname = 'GLAT', lonname = 'GLON', dtime = dt.datetime(year, 1, 1), mode='N2S')
        data = stations
        # add year column to table
        data = pd.DataFrame(data)
        data['Year'] = year
        # append table to big table
        # stations_over_time.concat(data)
        stations_over_time = pd.concat([stations_over_time, data])
        print('Appending to main table.........................................')
        print(stations_over_time.shape)
        # stations_over_time = pd.concat(data)
        
    except Exception as e:
        print(e)


# write DataFrame to an excel sheet 
stations_over_time
stations_over_time.to_csv('stations-over-time.csv')
stations_over_time.head()

In [ ]:
stations_over_time.shape

In [ ]:
import plotly.express as px

In [ ]:
# # Read .csv file:
# stations_over_time = pd.read_csv('stations-over-time.csv')

# If year < start or > end, delete line.
stations_over_time = stations_over_time[stations_over_time.Year <stations_over_time.End]
stations_over_time = stations_over_time[stations_over_time.Year >stations_over_time.Start]

# Create the animation
world_map = px.scatter_geo(stations_over_time.reset_index(), lon='GLON', lat='GLAT', 
                           color = 'Nation',
                           symbol = 'Network',
                           animation_frame='Year', animation_group='Year', projection='orthographic', hover_name='Name', hover_data = ['ID', 'Network'], 
                           basemap_visible = True
                          )
# world_map = px.scatter_geo(world, lon='long', lat='lat', size='mass', color='orange', animation_frame='year', animation_group='group', projection='orthographic', hover_name='name')
world_map.update_layout(title='Stations Over Time',  title_x=0.5, title_font=dict(size=10))

world_map.update_layout(
    geo = dict(
        showland = True,
        showcountries = True,
        showocean = True,
        countrywidth = 0.5,
        # landcolor = 'rgb(250, 155, 76)',
        # lakecolor = 'rgb(0, 255, 255)',
        # oceancolor = 'rgb(0, 255, 255)',
        projection = dict(
            type = 'orthographic',
            rotation = dict(
                lon = -100,
                lat = 40,
                roll = 0
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        ),
        lataxis = dict(
            showgrid = True,
            gridcolor = 'rgb(102, 102, 102)',
            gridwidth = 0.5
        )
    )
)

# fig.show()

# Display the animation
world_map.show()
world_map.write_html("animated-map.html")

In [ ]:
# # Read .csv file:
# stations_over_time = pd.read_csv('stations-over-time.csv')

# If year < start or > end, delete line.
stations_over_time = stations_over_time[stations_over_time.Year <stations_over_time.End]
stations_over_time = stations_over_time[stations_over_time.Year >stations_over_time.Start]

# Create the animation
world_map = px.scatter_geo(stations_over_time.reset_index(), lon='GLON', lat='GLAT', 
                           color = 'Nation',
                           symbol = 'Network',
                           facet_col='Year', projection='orthographic', hover_name='Name', hover_data = ['ID', 'Network'], 
                           basemap_visible = True
                          )
# world_map = px.scatter_geo(world, lon='long', lat='lat', size='mass', color='orange', animation_frame='year', animation_group='group', projection='orthographic', hover_name='name')
world_map.update_layout(title='Stations Over Time',  title_x=0.5)

# Get unique years for facet titles
unique_years = sorted(stations_over_time['Year'].unique())

# Iterate through all geo subplots to apply styling, centering, zooming, and facet titles
for i, year in enumerate(unique_years):
    world_map.update_geos(
        row=1, col=i + 1,  # Adjust col index to be 1-based
        showland=True,
        showcountries=True,
        showocean=True,
        countrywidth=0.5,
        projection=dict(
            type='orthographic',
            rotation=dict(
                lon=0,  # Centering longitude for Antarctica
                lat=-90, # Centering latitude for Antarctica
                roll=0
            ),
            scale=1.5 # Zoom in by setting scale greater than 1 (default is 1)
        ),
        lonaxis=dict(
            showgrid=True,
            gridcolor='rgb(102, 102, 102)',
            gridwidth=0.5
        ),
        lataxis=dict(
            showgrid=True,
            gridcolor='rgb(102, 102, 102)',
            gridwidth=0.5
        )
    )
    # Title each facet by year
    world_map.layout.annotations[i]['text'] = str(year)
    
# Display the animation
world_map.show()
world_map.write_html("faceted-map.html")